In [227]:
import numpy as np
import altair as alt
import pandas as pd
alt.themes.enable('dark')

ThemeRegistry.enable('dark')

In [228]:
number_of_rings = 4

hex_coords = []

for a in range(number_of_rings-1, -number_of_rings, -1):
    b_start = -number_of_rings+1+a if a > 0 else -number_of_rings+1
    b_end = number_of_rings-1 if a > 0 else number_of_rings-1+a
    for b in range(b_start, b_end+1):
        hex_coords.append((a, b))

In [229]:
number_of_rings = 5

a = 4
b_start = 0
while True:

    print(f"{a} {b_start}")

    if a == -number_of_rings+1:
        break
    if b_start > -number_of_rings+1:
        b_start -= 1

    a -= 1




4 0
3 -1
2 -2
1 -3
0 -4
-1 -4
-2 -4
-3 -4
-4 -4


In [246]:
sample_coords = np.array([[1, 0], [1,1], [0,1], [-1, 0], [-1, -1], [0, -1]])


def transform_ring_coords(coords, hex_width=1):
    a_coords = coords[:, 0] # the first dimension (-60 degrees from the x-axis)
    b_coords = coords[:, 1] # the second dimension (the x-axis)

    # Compute the x and y coordinates
    x_coords = -0.5 * hex_width * a_coords + hex_width * b_coords
    y_coords = (hex_width * np.sqrt(3) / 2) * a_coords

    # Stack x and y coordinates into a single array of shape (N, 2)
    converted_coords = np.column_stack((x_coords, y_coords))

    return converted_coords


def convert_coords(coords, h=1):
    # Extract a and b coordinates from the input array
    a_coords = coords[:, 0]
    b_coords = coords[:, 1]

    # Compute the x and y coordinates using the transformation formulas
    x_coords = (-h * np.sqrt(3) / 2) * a_coords
    y_coords = (-h / 2) * a_coords + h * b_coords

    # Stack x and y coordinates into a single array of shape (N, 2)
    converted_coords = np.column_stack((x_coords, y_coords))

    return converted_coords




def get_hex_coords(n, number_of_rings=None):
    hex_coords = []

    # how many rings are needed for n hexes? With n rings there are 3n^2 - 3n + 1 hexes
    #\frac{3+\sqrt{12x-3}}{6}
    if not number_of_rings:
        number_of_rings = np.ceil((3+np.sqrt(12*n-3))/6)
        print(f"{number_of_rings} needed for {n} hexes")

    for a in np.arange(number_of_rings-1, -number_of_rings, -1):
        b_start = -number_of_rings+1+a if a > 0 else -number_of_rings+1
        b_end = number_of_rings-1 if a > 0 else number_of_rings-1+a
        #for b in range(b_start, b_end+1):
        for b in np.arange(b_start, b_end+1):
            hex_coords.append((a, b))

    return np.array(hex_coords)[:n]

def get_ring_numbers(number_of_rings):
    ring_numbers = []

    existing_rings = np.array([])
    for a in np.arange(number_of_rings, 0, -1):
        ring_numbers.append((np.concatenate((existing_rings, np.array([a]).repeat(a), existing_rings[::-1]))))
        existing_rings = np.concatenate((existing_rings, np.array([a])))

    # Reverse the list excluding the last element
    reversed_part = ring_numbers[:-1][::-1]

    # Append the reversed part to the original list
    ring_numbers = ring_numbers + reversed_part

    ring_numbers = np.concatenate(ring_numbers)
    return ring_numbers


hex_coords = get_hex_coords(61, None)
cartesian_coords = convert_coords(hex_coords, 1)

print(len(hex_coords))

coords = hex_coords+cartesian_coords

df = pd.DataFrame(np.column_stack((cartesian_coords, hex_coords)), columns=['x', 'y', 'a', 'b']).reset_index()
df['ring'] = df['a'].pow(2) + df['b'].pow(2)

ring_numbers = get_ring_numbers(5)
df['ring_number'] = ring_numbers[:len(df)]


alt.Chart(df).mark_circle(size=50).encode(
    x='x',
    y='y',
    color=alt.Color('ring_number', type='nominal', scale=alt.Scale(scheme='set1')),
    detail='ring_number',
    tooltip=['x', 'y', 'a', 'b', 'ring','index',]
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
).properties(
    width=400,
    height=400
).interactive()





5.0 needed for 61 hexes
61


alt.Chart(...)

In [244]:
xyCoords = transform_ring_coords(hex_coords, hex_width=1)
xyCoords[-1]

array([ 2.        , -3.46410162])

In [231]:
number_of_rings = 5

ring_numbers = []

existing_rings = np.array([])
for a in np.arange(number_of_rings, 0, -1):
    ring_numbers.append((np.concatenate((existing_rings, np.array([a]).repeat(a), existing_rings[::-1]))))
    existing_rings = np.concatenate((existing_rings, np.array([a])))

# Reverse the list excluding the last element
reversed_part = ring_numbers[:-1][::-1]

# Append the reversed part to the original list
ring_numbers = ring_numbers + reversed_part

ring_numbers = np.concatenate(ring_numbers)
ring_numbers

array([5., 5., 5., 5., 5., 5., 4., 4., 4., 4., 5., 5., 4., 3., 3., 3., 4.,
       5., 5., 4., 3., 2., 2., 3., 4., 5., 5., 4., 3., 2., 1., 2., 3., 4.,
       5., 5., 4., 3., 2., 2., 3., 4., 5., 5., 4., 3., 3., 3., 4., 5., 5.,
       4., 4., 4., 4., 5., 5., 5., 5., 5., 5.])

In [206]:
ring_numbers

[array([5., 5., 5., 5., 5.]),
 array([5., 4., 4., 4., 4., 5.]),
 array([5., 4., 3., 3., 3., 4., 5.]),
 array([5., 4., 3., 2., 2., 3., 4., 5.]),
 array([5., 4., 3., 2., 1., 2., 3., 4., 5.])]

In [203]:
type(ring_numbers)

list

In [200]:
ring_numbers

[array([5., 5., 5., 5., 5.]),
 array([5., 4., 4., 4., 4., 5.]),
 array([5., 4., 3., 3., 3., 4., 5.]),
 array([5., 4., 3., 2., 2., 3., 4., 5.]),
 array([5., 4., 3., 2., 1., 2., 3., 4., 5.])]

In [ ]:
(1,2,3)

In [ ]:
existing_rings
np.array([a]).repeat(a)
existing_rings[::-1]
np.concatenate(existing_rings, np.array([a]).repeat(a))

In [ ]:
np.concatenate((existing_rings, np.array([a]).repeat(a), existing_rings[::-1])) 

In [ ]:
existing_rings[::-1]

In [ ]:
np.array([a]).repeat(a)

In [ ]:
existing_rings

In [ ]:
np.array

In [ ]:
np.array([8]).repeat(8)

In [ ]:
np.array([1,2,3,]).reverse()

In [ ]:
[5]*4

In [ ]:
len(hex_coords)

In [ ]:
len(get_hex_coords(None, 5))

In [ ]:
get_hex_coords(60)

In [ ]:
df

In [ ]:
np.arange(-4, 0+1)

In [ ]:
len(cartesian_coords)

In [ ]:
cartesian_coords[:56]

In [ ]:
np.arange(4, -5, -1)

In [ ]:
hex_coords

In [ ]:
hex_coords = get_hex_coords(4)

np.ceil(4+5/6)

In [ ]:
(4+5)/6